# Creating a new Weather Model in RAiDER

**Author**: Jeremy Maurer, David Bekaert, Simran Sangha, Yang Lei - Jet Propulsion Laboratory, California Institute of Technology

This notebook provides an overview of how to get started using the RAiDER package for estimating tropospheric RADAR delays, and other functionality included in the **raiderDelay.py** program. We give an example of how to download and process delays using ERA-5 and HRRR weather models for the Los Angeles region. 

In this notebook, we will demonstrate how to:
- Define and use a custom weather model for use with RAiDER
    
<div class="alert alert-warning">
The initial setup (<b>Prep A</b> section) should be run at the start of the notebook. 
</div>

<div class="alert alert-danger">
<b>Potential Errors:</b> 
    - RAiDER needs to be installed to run this notebook
</div>


<div class="alert alert-info">
    <b>Terminology:</b>
    
- *Weather model*: A reanalysis weather product defining temperature, pressure, and humidity on a regular grid in some coordinate system (e.g., at regular lat/lon intervals in the standard EPSG:4326 reference frame).
</div>
    

## Some initial setup 

In [ ]:
from osgeo import gdal
import os
import numpy as np
import matplotlib.pyplot as plt

## Defining the home and data directories at the processing location
work_dir = os.path.abspath(os.getcwd())
tutorial_home_dir = os.path.abspath(os.getcwd())
print("Work directory: ", work_dir)
print("Tutorial directory: ", tutorial_home_dir)

# Enable GDAL/OGR exceptions
gdal.UseExceptions()

# Verifying if ARIA-tools is installed correctly
try:
    import RAiDER
except:
    raise RuntimeError('RAiDER is missing from your PYTHONPATH')

os.chdir(work_dir)

# RAiDER Readers

Weather model readers provide the link between the raw weather model data (e.g. available from ECMWF, ERA-5, ERA-5T, GMAO, MERRA-2, HRRR), and the absolute delay calculation. Readers can be added by users to account for other models and custom formats. Here we provide an overview of the WeatherModel class object and requirements for writing one's own reader function. 

## The WeatherModel class

### Functions to be overloaded:
\_fetch: 
- Called by WeatherModel.fetch method
- downloads or loads data from the source files

load_weather: 
- Called by the WeatherModel.load method
- loads data from the raw weather model files and parses it into the WeatherModel format (see below)

### Defining a custom Reader
The example below describes the minimum required attributes and methods for a custom model reader. Each model reader should call as a super-class the "WeatherModel" base class and should initialize the base class as shown in the example. This will initialize all of the required attributes, etc. and default values for non-required attributes.  

The minimum required class methods are ```__init__```, ```_fetch``` and ```load_weather```, and auxiliary methods and attributes can be defined as needed for accessing the data API, loading and manipulating the data, etc. 

#### Required data and format
RAiDER expects that your custom weather model reader will result in a Python object with attributes consistent with the WeatherModel class and the RAiDER convention. The required variables are: 
- \_lats, \_lons
- \_p, \_t
- \_rh OR \_q, matching the corresponding \_humidityType

In addition, you need three variables that capture the coordinates of the data cubes:
- \_xs, \_ys, \_zs

Each of the required variables should be a 3-D cube, all of the same shape, with axes ordered as (z, x, y), monotonically increasing***. \_lons and \_lats should also be 3D cubes, replicated in the z-dimension. 

<div class="alert alert-warning">
    
The longitude '_lons' needs to vary between -180 and 180 (longitudes between 0 and 360 are not supported).
    

The '_zs' variable should be topographic height, but the height variable passed with the weather model data is often the geopotential height, which must be converted to topographic height. The WeatherModel class has a helper function for this conversion, which can be called within the custom class as self._get_heights(lats, geo_hgt), where geo_hgt is geopotential height. 
</div>

In [ ]:
from RAiDER.models.weatherModel import WeatherModel

class customModelReader(WeatherModel):
    def __init__(self):
        WeatherModel.__init__(self)
        
        # **Note**: The equation for refractivity uses e, ***<def>, but
        # typically weather models provide either q (specific humidity) or
        # rh (relative humidity). RAiDER computes e automatically from
        # either of these.
        self._humidityType = 'q'  # can be "q" (specific humidity) or "rh" (relative humidity)

        # This is useful if a single weather model provides data on both fixed
        # pressure levels and fixed model levels (e.g., ECMWF). You can define
        # different readers for both types
        self._model_level_type = 'pl' # Default, pressure levels are "pl", and model levels are "ml"

        # Tuple of min/max years where data is available.
        # valid range of the dataset. Users need to specify the start date and
        # end date (can be "present")
        self._valid_range = (datetime.datetime(2016,7,15),"Present")

        # Lag time between today and when today's data will be available for
        # download.
        # Can be specified in hours "hours=3" or in days "days=3"
        self._lag_time = datetime.timedelta(hours=3)

        # model constants (these three borrowed from the ECMWF model)
        # These the k's in the expression for refractivity:
        #   k1*(P/T) + k2*(e/T) + k3*(e/T^2)
        self._k1 = 0.776  # [K/Pa]
        self._k2 = 0.233 # [K/Pa]
        self._k3 = 3.75e3 # [K^2/Pa]

        # horizontal grid spacing. These are NOT used for projection
        # information, but are used to estimate a buffer region around your
        # query points to ensure that a large enough area is downloaded
        self._lat_res = 3./111  #  grid spacing in latitude
        self._lon_res = 3./111  #  grid spacing in longitude
        self._x_res = 3.        #  x-direction grid spacing in the native weather model projection
                                #  (if the projection is in lat/lon, it is the same as "self._lon_res")
        self._y_res = 3.        #  y-direction grid spacing in the weather model native projection
                                #  (if the projection is in lat/lon, it is the same as "self._lat_res")

        self._Name = 'ABCD' #  name of the custom weather model (better to be capitalized)

        # Projections in RAiDER are defined using pyproj
        # (python wrapper around Proj)
        # If the projection is defined with EPSG code, one can use
        #   "self._proj = CRS.from_epsg(4326)"
        # to replace the following lines to get "self._proj".
        # Below we show the example of HRRR model with the parameters of its
        # Lambert Conformal Conic projection
        lon0 = 262.5
        lat0 = 38.5
        lat1 = 38.5
        lat2 = 38.5
        x0 = 0
        y0 = 0
        earth_radius = 6371229
        self._proj = CRS(
            '+proj=lcc ' + \
            f'+lat_1={lat1} +lat_2={lat2} +lat_0={lat0} +lon_0={lon0} ' + \
            f'+x_0={x0} +y_0={y0} +a={earth_radius} +b={earth_radius} ' + \
            '+units=m +no_defs'
        )


    # This function needs to be writen by the users and is used to e.g. download
    # a file containing weather data (p, t, rh, etc.) from the weather model
    # server, or iff your weather models always live locally, you can define
    # logic here to read a subset of the files based on the input bounding box.
    def _fetch(self, lats, lons, time, out):
        '''
        Fetch weather model data from the custom weather model "ABCD"

        Parameters
        ----------
        NDArray:                lats - latitude of your query points
        NDArray:                lons - longitude of your query points
        Python Datetime object: time - datatime object (year, month, day, hour, minute, second)
        String:                 out  - name of downloaded dataset file from the custom weather model server
        '''
        # The list of inputs is exact; RAiDER will not pass any additional
        # keyword arguments to this function, and all of the inputs must be
        # provided.
        
        # bounding box plus a buffer using the helper function from the WeatherModel base class
        # 
        # Set the "Nextra" argument to match the number of additional grid cells
        # in your custom model to download outside of your query points. This is
        # needed when ray-tracing slant delays for the points on the edge.
        #
        # Nextra should be something like
        #   ceil(zref*tan(inc)/horizontal_grid_spacing),
        # where zref is the assumed height of the troposphere (default 15 km),
        # inc is the average inclination angle, and
        # horizontal_grid_spacing is from your model in km.
        # 
        # Example: Sentinel-1 (inc ~ 35 degrees), ERA-5 (grid spacing ~ 30 km)
        # and the default zref (15 km),
        # Nextra = 1.
        lat_min, lat_max, lon_min, lon_max = self._get_ll_bounds(lats, lons, Nextra = 1)
        self._bounds = (lat_min, lat_max, lon_min, lon_max)
        
        # Even if you don't need to download files, you will need to assign the
        # "self._files" attribute so that the "load_weather" method knows what
        # file contains the data
        # 
        # In this example, you would need to define an auxilliary function
        # _download_abcd_file (see below)
        self._files = self._download_abcd_file(out, time, self._bounds)


    # This function gets called by RAiDER to read individual variables from your
    # source file and pre-process the data from the file into the format
    # expected by RAiDER (see main text above and "Returns" description below).
    def load_weather(self, filename: str) -> None:
        '''
        Load weather model variables from the downloaded file named filename

        Parameters
        ---------- 
        filename - filename of the downloaded weather model file

        Returns
        -------
        # Doesn't directly return anything, but assigns values to self.
        # Data cubes: should be ordered as (z, x, y)
        NDArray: _p    - 3D data cube of pressure in Pa
        NDArray: _t    - 3D data cube of temperature in Kelvin
        NDArray: _q    - 3D data cube of specific humidity in ***; only one of _q or _rh is required
        NDArray: _rh   - 3D data cube of relative humidity in ***; only one of _q or _rh is required
        NDArray: _lats - 3D data cube of latitude. Should be WGS-84 latitudes (EPSG: 4326)
        NDArray: _lons - 3D data cube of longitude. Should be WGS-84 latitudes (EPSG: 4326)
        NDArray: _xs   - 3D cube of x-coordinates of the data cubes in the native weather model projection
        NDArray: _ys   - 3D cube of y-coordinates of the data cubes in the native weather model projection
        NDArray: _zs   - 3D cube of z-coordinates of the data cubes in the native weather model projection

        '''
        # In this case we have an auxiliary function "_makeDataCubes" to do the
        # pre-processing. Pre-processing loads the data available from the
        # weather model file and manipulates it as needed to get the data cubes
        # into the form prescribed above.
        lats, lons, xs, ys, t, q, p, hgt = self._makeDataCubes(filename)
        
        # **Note**: RAiDER provides helper functions for certain types of
        # operations; e.g. for converting surface pressure and geopotential to
        # pressure and geopotential height:
        #   z, p, hgt = self._calculategeoh(z, lnsp),
        # where z is geopotential and lnsp is the natural log of surface pressure

        # **Note**: ECMWF provides heights as geopotential (units m^2/s^2).
        # For a similar custom model, one can use the following line to convert
        # to geopotential height:
        #   hgt = z / self._g0
        
        # if geopotential height is provided, one can use the following line to
        # convert to topographic height, which is then automatically assigned to
        # self._zs:
        #   self._get_heights(lats, hgt)
        # where hgt is geopotential height = geopotential / gravity acceleration
        
        # Otherwise, if topographic height is provided directly:
        _zs = hgt
        
        # depending
        self._t = t
        self._q = q
        self._p = p
        self._lats = lats
        self._lons = lons

        # _xs: x-direction grid coordinate in the native weather model projection (=_lons if projection is WGS-84)
        # _ys: y-direction grid coordinate in the native weather model projection (=_lats if projection is WGS-84)
        # _zs: z-direction grid coordinate. Must be topographic height in meters.
        self._xs = xs
        self._ys = ys
        self._zs = _zs
        ###########

    def _download_abcd_file(self, out, date_time, bounding_box):
        '''
        Example auxilliary function for fetching data

        Can be a file download from a server, grabbing a local filename, or
        accessing a cloud-based API

        Parameters
        ----------
        out          - filename to save data to
        date_time    - Python datatime object
        bounding_box - bounding box for the region of interest 

        Output: 
        out - returned filename from input
        '''
        return None

    
    def _makeDataCubes(self, filename):
        '''
        Example auxilliary function for data pre-processing
        
        Read 3-D data cubes from 'filename' 
        
        Parameters
        ----------
        filename - filename of the downloaded weather model file from the server

        Returns
        -------
        lats - latitude (3-D data cube)
        lons - longitude (3-D data cube)
        xs - x-direction grid dimension of the native weather model coordinates (3-D data cube; if in lat/lon, _xs = _lons)
        ys - y-direction grid dimension of the native weather model coordinates (3-D data cube; if in lat/lon, _ys = _lats)
        t - temperature (3-D data cube)
        q - humidity (3-D data cube; could be relative humidity or specific humidity)
        p - pressure level (3-D data cube; could be pressure level (preferred) or surface pressure)
        hgt - height (3-D data cube; could be geopotential height or topographic height (preferred))
        '''
        return None, None, None, None, None, None, None, None
    

### The ```_fetch``` method
The ```_fetch``` method gets called by the RAiDER to fetch download or read the data. As shown in the example script, this is where you can download the data from a server, etc. If your weather model always lives on your local machine (or can always be locally accessed) this method can be very simple, but should still be defined as it will always be called. In addition, the filename of the data should be returned so that RAiDER knows what file to load. 

### The ```load_weather``` method
```load_weather``` is like ```_fetch``` in that it always gets called during the "load" routine. After you have a file available for RAiDER to read, this method will pre-process the data from your file to match the inputs RAiDER expects. In particular, after running this method, your weather model reader object should contain the variables listed in the example above. 

 ### Adding the reader to the weather model list

Modify the allowed list of weather models "allowed.py" under the directory of "tools/RAiDER/models" to include the custom "ABCD" model as below. 

In [ ]:
ALLOWED_MODELS = [
    'ERA5',
    'ERA5T',
    'ERAI',
    'MERRA2',
    'WRF',
    'HRRR',
    'GMAO',
    'HDF5',
    'HRES',
    'NCMR',
    'ABCD'
]

### Debugging your custom reader
The WeatherModel class has two built-in plots for debugging purposes:

```WeatherModel.plot(plotType='pqt', savefig=True)```

```WeatherModel.plot(plotType='wh', savefig=True)```

These commands plot pressure/humidity/temperature and wet and hydrostatic refractivity for the weather model, and are created by default when running ```raiderDelay.py``` normally.

When debugging your custom model reader, you can use the command line executable ```raiderWeatherModelDebug.py```, which can take the exactly same list of input variables as ```raiderDelay.py``` and just create the debugging plots.

In [ ]:
# Replace "ABCD" with your custom weather model name
# add the --out option if you want your results to be written to a directory other than the current one
raiderWeatherModelDebug.py --date 20200103 --time 23:00:00 -b 39 40 -79 -78 --model ABCD --zref 15000 -v

You can also test your custom model reader by running the three example commands from the ```raiderDelay.py``` helper message (i.e. running ```raiderDelay.py``` with the ```-h``` option will show the three example commands) with the weather model name "ERA5" replaced with the newly-added custom one, e.g. "ABCD".

In [ ]:
# Replace "ABCD" with your custom weather model name
# add the --out option if you want your results to be written to a directory other than the current one
raiderDelay.py --date 20200103 --time 23:00:00 -b 39 40 -79 -78 --model ABCD --zref 15000 -v

RAiDER has several features to aid in debugging custom weather models. 

The debugging plots have been generated for several weather models (ERA-5, ERA-5T, ERA-I, MERRA-2, GMAO, HRRR, ECMWF HRES, NCMR) on July 1st, 2018 at 00:00:00. ERA-5 is shown below, and the others are in the img directory in this folder.  

For each weather model, the first debugging plot (with option 'pqt') shows the pressure/humidity/temperature at an altitude of 500 m (top row) and 15,000 m (middle row), and then the bottom row shows the vertical variation of pressure/humidity/temperature located specifically at the black point (Los Angeles).

The second debugging plot (with option 'wh') shows the wet and hydrostatic refractivity at the altitude of 500 m (top row) and 15,000 m (bottom row).

![ERA-5 Weather Model Data 20180701T00:00:00](img/ERA-5_weather_hgt500_and_15000m.png)
![ERA-5 Refractivity 20180701T00:00:00](img/ERA-5_refractivity_hgt500_and_15000m.png)